### Questions to be answered
How often should the second crew be called out?  
What are the expected monthly demurrage costs?  
If the standard trains could be scheduled to arrive at precise times, what daily schedule would
minimize loading costs?  
Would a third tipple-loading crew at $12,000 per hour reduce annual operations?  
Can this tipple support a fourth standard train every day?  

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
df=pd.read_csv("./schedule.csv")
df.head()

,TrainA,TrainB,TrainC,TrainHC
0,20,8,17,-1
1,13,12,19,-1
2,13,8,12,-1
3,8,11,7,-1
4,10,7,13,12


In [3]:
df.describe()

,TrainA,TrainB,TrainC,TrainHC
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,12.582200,12.553300,12.488900,0.852200
std,4.620604,4.597268,4.585824,4.548849
min,5.000000,5.000000,5.000000,-1.000000
25%,9.000000,9.000000,9.000000,-1.000000
50%,13.000000,13.000000,13.000000,-1.000000
75%,17.000000,17.000000,16.000000,-1.000000
max,20.000000,20.000000,20.000000,13.000000


In [4]:
df['Crews']=''
df['Crew Wages']=''
df['Demurrage Fees']=''
df['Total Cost']=''

In [5]:
def mine_cost(crews):
    return ((12000*crews)-3000)

def mine_rate(crews):
    return .25*crews

def load_cost(crews):
    return ((12000*crews)-3000)

def load_rate(crews):
    return .33*crews

def demurrage_cost(trains):
    return ((trains[0]*15000)+(trains[1]*25000))

### 3 train simulation code

In [6]:
tipple=load=0

for x in range(0,len(df)):
    tipples=loads=hc_load=daily_loads=crew_cost=demurrage=0
    a=df.loc[x,'TrainA']
    b=df.loc[x,'TrainB']
    c=df.loc[x,'TrainC']
    hc=df.loc[x,'TrainHC']
    
    crews=1
    max_crews=1
    capacity=1.5
    
    trains=[a,b,c,hc]
    arrived=[]
    departed=[]
    demurrage_rate=[0,0]
    trains=np.sort(trains)
    
    # Get rid of placeholder value for TrainHC
    if x%7==4:
        trains=np.delete(trains,0)
        
    for i in range(0,24):
        # A train has arrived, add it to the list of arrivals
        if i in trains:
            arrived.append(i)            
            # Check to see if train is HC
            if i==hc:
                # Check to see if other trains are still being loaded
                crews=2
                max_crews=2
                # Tipple is full, unload
                if tipple==capacity:
                    hc_load+=load_rate(crews)
                    tipple-=load_rate(crews)
                    crew_cost+=load_cost(crews)
                    if demurrage_rate[0]>0:
                        demurrage+=demurrage_cost(demurrage_rate)

                    if hc_load>=2:
                        departed.append(hc)
                        arrived.remove(hc)
                        tipple+=hc_load-2
                        demurrage_rate[1]=0
                # Enough coal for HC, start loading
                elif tipple+hc_load>=2:
                    hc_load+=load_rate(crews)
                    tipple-=load_rate(crews)
                    crew_cost+=load_cost(crews)
                    # add demurrage for normal trains waiting
                    if demurrage_rate[0]>0:
                        demurrage+=demurrage_cost(demurrage_rate)
                    # hc train full, depart
                    if hc_load>=2:
                        departed.append(hc)
                        arrived.remove(hc)
                        tipple+=hc_load-2
                        # Set hc demurrage flag to 0
                        demurrage_rate[1]=0
                # Not enough in tipple for hc, mine more
                else:
                    # hc demurrage flag to 1
                    demurrage_rate[1]=1
                    tipple+=mine_rate(crews)
                    crew_cost+=mine_cost(crews)
                    demurrage+=demurrage_cost(demurrage_rate)
                

            
            # Not HC train
            else:
                # More than 1 train is in, add demurrage and keep loading
                if (len(arrived)>1):
                    # More then one train waiting to be loaded
                    crews=2
                    max_crews=2
                    # HC train check
                    if hc in arrived and hc not in departed:
                        # Tipple is full, unload
                        if tipple==capacity:
                            hc_load+=load_rate(crews)
                            tipple-=load_rate(crews)
                            crew_cost+=load_cost(crews)
                            # Calculate demurrage for waiting trains
                            if demurrage_rate[0]>0:
                                demurrage+=demurrage_cost(demurrage_rate)
                            # HC train full
                            if hc_load>=2:
                                departed.append(hc)
                                arrived.remove(hc)
                                tipple+=hc_load-2
                                # Reset hc demurrage flag
                                demurrage_rate[1]=0
                        # Enough coal for HC, start loading
                        elif tipple+hc_load>=2:
                            hc_load+=load_rate(crews)
                            tipple-=load_rate(crews)
                            crew_cost+=load_cost(crews)
                            if demurrage_rate[0]>0:
                                demurrage+=demurrage_cost(demurrage_rate)

                            if hc_load>=2:
                                departed.append(hc)
                                arrived.remove(hc)
                                tipple+=hc_load-2
                                demurrage_rate[1]=0                        
                        # Not enough coal for HC
                        else:
                            # Mine more and set hc demurrage flag to 1
                            demurrage_rate[1]=1
                            crew_cost+=mine_cost(crews)
                            tipple+=mine_rate(crews)
                            demurrage+=demurrage_cost(demurrage_rate)

                                
                    # HC not waiting
                    else:
                        # Enough coal to fill train
                        if tipple+load>=1:
                            if demurrage_rate[0]>0:
                                demurrage+=demurrage_cost(demurrage_rate)
                            load+=load_rate(crews)
                            tipple-=load_rate(crews)
                            crew_cost+=load_cost(crews)
                            
                            if load>=1:
                                #print("train full")
                                departed.append(min(arrived))
                                arrived.remove(min(arrived))
                                #print("departed", departed, "arrived", arrived)
                                tipple+=load-1
                                if demurrage_rate[0]>=1:
                                    demurrage_rate[0]-=1
                                load=0                        
                        # Not enough coal, mine more
                        else:
                            demurrage_rate[0]+=(len(arrived))
                            demurrage+=demurrage_cost(demurrage_rate)
                            tipple+=mine_rate(crews)
                            crew_cost+=mine_cost(crews)

                            
                # Only one train to load
                else:
                    crews=1
                    # Enough coal in tipple to load, load
                    if tipple+load>=1:
                        if demurrage_rate[0]>0:
                                demurrage+=demurrage_cost(demurrage_rate)
                        load+=load_rate(crews)
                        tipple-=load_rate(crews)
                        crew_cost+=load_cost(crews)
                        if demurrage_rate[0]>=1:
                            demurrage_rate[0]-=1
                            demurrage+=demurrage_cost(demurrage_rate)
                        if load>=1:
                            departed.append(min(arrived))
                            arrived.remove(min(arrived))
                            tipple+=load-1
                            load=0
                    # Not enough coal to load current train, mine more
                    else:
                        crews=2
                        max_crews=2
                        demurrage_rate[0]+=(len(arrived))
                        demurrage+=demurrage_cost(demurrage_rate)
                        tipple+=mine_rate(crews)
                        crew_cost+=mine_cost(crews)

                       
        # No new trains have arrived
        else:
            # Check status of arrived trains
            if arrived:
                # Train exists, check HC status
                if hc in arrived and hc not in departed:
                    crews=2
                    max_crews=2
                    if tipple==capacity:
                        hc_load+=load_rate(crews)
                        tipple-=load_rate(crews)
                        crew_cost+=load_cost(crews)
                        demurrage_rate[0]=(len(arrived)-1)
                        demurrage+=demurrage_cost(demurrage_rate)
                        if hc_load>=2:
                            if demurrage_rate[0]>=1:
                                demurrage_rate[0]-=1
                            departed.append(hc)
                            arrived.remove(hc)
                            tipple+=hc_load-2
                            demurrage_rate[1]=0
                    # Enough coal for HC, start loading
                    elif tipple+hc_load>=2:
                        hc_load+=load_rate(crews)
                        tipple-=load_rate(crews)
                        crew_cost+=load_cost(crews)
                        demurrage_rate[0]=(len(arrived)-1)
                        demurrage+=demurrage_cost(demurrage_rate)
                        if hc_load>=2:
                            if demurrage_rate[0]>=1:
                                demurrage_rate[0]-=1
                            departed.append(hc)
                            arrived.remove(hc)
                            tipple+=hc_load-2
                            demurrage_rate[1]=0                            
                    # Not enough coal for HC
                    else:
                        # Mine more and set hc demurrage flag to 1
                        demurrage_rate[0]=(len(arrived)-1)
                        demurrage_rate[1]=1
                        crew_cost+=mine_cost(crews)
                        tipple+=mine_rate(crews)
                        demurrage+=demurrage_cost(demurrage_rate)
                        

                                
                # HC not waiting
                else:
                    if tipple+load>=1:
                        demurrage_rate[0]=(len(arrived)-1)
                        demurrage+=demurrage_cost(demurrage_rate)
                        load+=load_rate(crews)
                        tipple-=load_rate(crews)
                        crew_cost+=load_cost(crews)
 
                        if load>=1:
                            if demurrage_rate[0]>=1:
                                demurrage_rate[0]-=1 
                            departed.append(min(arrived))
                            arrived.remove(min(arrived))
                            tipple+=load-1
                            load=0                    
                    else:
                        crews=2
                        max_crews=2
                        demurrage_rate[0]=(len(arrived))
                        demurrage+=demurrage_cost(demurrage_rate)
                        tipple+=mine_rate(crews)
                        crew_cost+=mine_cost(crews)

                    
            # If none, keep mining
            else:
                crews=1
                if tipple<capacity:
                    tipple+=mine_rate(crews)
                    crew_cost+=mine_cost(crews)
                    
    df.loc[x,'Crews']=max_crews
    df.loc[x,'Crew Wages']=crew_cost
    df.loc[x,'Demurrage Fees']=demurrage
    df.loc[x,'Total Cost']=(crew_cost+demurrage)

### 4 train simulation code

In [7]:
# coal=load=0

# for x in range(0,len(df)):
#     loads=hc_load=daily_loads=crew_cost=demurrage=0
#     a=df.loc[x,'TrainA']
#     b=df.loc[x,'TrainB']
#     c=df.loc[x,'TrainC']
#     d=df.loc[x,'TrainD']
#     hc=df.loc[x,'TrainHC']
    
#     crews=1
#     max_crews=1
    
#     trains=[a,b,c,hc]
#     arrived=[]
#     departed=[]
#     demurrage_rate=[0,0]
#     trains=np.sort(trains)

#     if x%7==4:
#         max_loads=6
#     else:
#         max_loads=4
#         # Get rid of placeholder value for TrainHC
#         trains=np.delete(trains,0)
        
#     for i in range(0,24):
#         #print(crew_cost,coal,arrived,departed,demurrage)
#         # A train has arrived, add it to the list of arrivals
#         if i in trains:
#             arrived.append(i)
#             #print(i, "arrives")
            
#             # Check to see if train is HC
#             if i==hc:
#                 #print("is high capacity")
#                 # Check to see if other trains are still being loaded
#                 crews=2
#                 max_crews=2
#                 # Not enough coal for HC
#                 if coal+hc_load<2:
#                     #print("need coal")
#                     # Mine more and set hc demurrage flag to 1
#                     demurrage_rate[1]=1
#                     coal+=mine_rate(crews)
#                     crew_cost+=mine_cost(crews)
#                     demurrage+=demurrage_cost(demurrage_rate)
#                 # Enough coal for HC, start loading
#                 else:
#                     #print("have coal, loading")
#                     hc_load+=load_rate(crews)
#                     coal-=load_rate(crews)
#                     crew_cost+=load_cost(crews)
#                     if demurrage_rate[0]>0:
#                         demurrage+=demurrage_cost(demurrage_rate)
                    
#                     if hc_load>=2:
#                         #print("hc loaded")
#                         departed.append(hc)
#                         #print("added hc to departed",departed)
#                         arrived.remove(hc)
#                         #print("removed hc from departed" ,arrived)
#                         coal+=hc_load-2
#                         demurrage_rate[1]=0
            
#             # Not HC train
#             else:
#                 #print("not high capacity")
#                 # More than 1 train is in, add demurrage and keep loading
#                 if (len(arrived)>1):
#                     #print("not only train in queue")
#                     # More then one train waiting to be loaded
#                     crews=2
#                     max_crews=2
#                     # HC train check
#                     if hc in arrived and hc not in departed:
#                         #print("other train high capacity")
#                         # Not enough coal for HC
#                         if coal+hc_load<2:
#                             #print("need more coal")
#                             # Mine more and set hc demurrage flag to 1
#                             demurrage_rate[1]=1
#                             crew_cost+=mine_cost(crews)
#                             coal+=mine_rate(crews)
#                             demurrage+=demurrage_cost(demurrage_rate)
#                         # Enough coal for HC, start loading
#                         else:
#                             #print("loading high capacity")
#                             hc_load+=load_rate(crews)
#                             coal-=load_rate(crews)
#                             crew_cost+=load_cost(crews)
#                             if demurrage_rate[0]>0:
#                                 demurrage+=demurrage_cost(demurrage_rate)

#                             if hc_load>=2:
#                                 #print("hc full")
#                                 departed.append(hc)
#                                 #print("added hc to departed",departed)
#                                 arrived.remove(hc)
#                                 #print("removed hc from departed" ,arrived)
#                                 coal+=hc_load-2
#                                 demurrage_rate[1]=0
                                
#                     # HC not waiting
#                     else:
#                         #print("train not high capacity")
#                         if coal+load<1:
#                             #print("not enough coal")
#                             demurrage_rate[0]+=(len(arrived))
#                             demurrage+=demurrage_cost(demurrage_rate)
#                             coal+=mine_rate(crews)
#                             crew_cost+=mine_cost(crews)
#                         else:
#                             #print("loading train")
#                             if demurrage_rate[0]>0:
#                                 demurrage+=demurrage_cost(demurrage_rate)
#                             load+=load_rate(crews)
#                             coal-=load_rate(crews)
#                             crew_cost+=load_cost(crews)
                            
#                             if load>=1:
#                                 #print("train full")
#                                 departed.append(min(arrived))
#                                 arrived.remove(min(arrived))
#                                 #print("departed", departed, "arrived", arrived)
#                                 coal+=load-1
#                                 if demurrage_rate[0]>=1:
#                                     demurrage_rate[0]-=1
#                                 load=0
                            
#                 # Only one train to load
#                 else:
#                     #print("only train")
#                     crews=1
#                     if coal+load<1:
#                         #print("getting more coal")
#                         crews=2
#                         max_crews=2
#                         demurrage_rate[0]+=(len(arrived))
#                         demurrage+=demurrage_cost(demurrage_rate)
#                         coal+=mine_rate(crews)
#                         crew_cost+=mine_cost(crews)
#                     else:
#                         #print("loading")
#                         if demurrage_rate[0]>0:
#                                 demurrage+=demurrage_cost(demurrage_rate)
#                         load+=load_rate(crews)
#                         coal-=load_rate(crews)
#                         crew_cost+=load_cost(crews)
#                         if demurrage_rate[0]>=1:
#                             demurrage_rate[0]-=1
#                             demurrage+=demurrage_cost(demurrage_rate)
#                         if load>=1:
#                             departed.append(min(arrived))
#                             arrived.remove(min(arrived))
#                             #print("departed", departed, "arrived", arrived)
#                             coal+=load-1
#                             load=0
                       
#         # No new trains have arrived
#         else:
#             #print("No new trains this hour")
#             # Check status of arrived trains
#             if arrived:
#                 #print("train already in queue")
#                 # Train exists, check HC status
#                 if hc in arrived and hc not in departed:
#                     #print("is high capacity, doubling crews")
#                     crews=2
#                     max_crews=2
#                     # Not enough coal for HC
#                     if coal+hc_load<2:
#                         #print("need more coal")
#                         # Mine more and set hc demurrage flag to 1
#                         demurrage_rate[0]=(len(arrived)-1)
#                         demurrage_rate[1]=1
#                         crew_cost+=mine_cost(crews)
#                         coal+=mine_rate(crews)
#                         demurrage+=demurrage_cost(demurrage_rate)
#                     # Enough coal for HC, start loading
#                     else:
#                         #print("loading high capacity")
#                         hc_load+=load_rate(crews)
#                         coal-=load_rate(crews)
#                         crew_cost+=load_cost(crews)
#                         demurrage_rate[0]=(len(arrived)-1)
#                         demurrage+=demurrage_cost(demurrage_rate)
#                         if hc_load>=2:
#                             #print("hc loaded")
#                             if demurrage_rate[0]>=1:
#                                 demurrage_rate[0]-=1
#                             departed.append(hc)
#                             #print("added hc to departed",departed)
#                             arrived.remove(hc)
#                             #print("removed hc from departed" ,arrived)
#                             coal+=hc_load-2
#                             demurrage_rate[1]=0
                                
#                 # HC not waiting
#                 else:
#                     #print("normal train waiting")
#                     if coal+load<1:
#                         #print("not enough coal")
#                         crews=2
#                         max_crews=2
#                         demurrage_rate[0]=(len(arrived))
#                         demurrage+=demurrage_cost(demurrage_rate)
#                         coal+=mine_rate(crews)
#                         crew_cost+=mine_cost(crews)
#                     else:
#                         #print("loading train")
#                         demurrage_rate[0]=(len(arrived)-1)
#                         demurrage+=demurrage_cost(demurrage_rate)
#                         load+=load_rate(crews)
#                         coal-=load_rate(crews)
#                         crew_cost+=load_cost(crews)
 
#                         if load>=1:
#                             #print("train full")
#                             if demurrage_rate[0]>=1:
#                                 demurrage_rate[0]-=1 
#                             departed.append(min(arrived))
#                             arrived.remove(min(arrived))
#                             #print("departed", departed, "arrived", arrived)
#                             coal+=load-1
#                             load=0
                    
#             # If none, keep mining
#             else:
#                 crews=1
#                 if coal<max_loads:
#                     coal+=mine_rate(crews)
#                     crew_cost+=mine_cost(crews)
                    
#     df.loc[x,'Crews']=max_crews
#     df.loc[x,'Crew Wages']=crew_cost
#     df.loc[x,'Demurrage Fees']=demurrage
#     df.loc[x,'Total Cost']=(crew_cost+demurrage)

In [8]:
df.head()

,TrainA,TrainB,TrainC,TrainHC,Crews,Crew Wages,Demurrage Fees,Total Cost
0,20,8,17,-1,2,246000,15000,261000
1,13,12,19,-1,2,270000,60000,330000
2,13,8,12,-1,2,264000,75000,339000
3,8,11,7,-1,2,261000,90000,351000
4,10,7,13,12,2,270000,345000,615000


In [9]:
df.describe()

,TrainA,TrainB,TrainC,TrainHC
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,12.582200,12.553300,12.488900,0.852200
std,4.620604,4.597268,4.585824,4.548849
min,5.000000,5.000000,5.000000,-1.000000
25%,9.000000,9.000000,9.000000,-1.000000
50%,13.000000,13.000000,13.000000,-1.000000
75%,17.000000,17.000000,16.000000,-1.000000
max,20.000000,20.000000,20.000000,13.000000


In [10]:
df['Total Cost'].mean()

304182.8

In [11]:
df['Demurrage Fees'].mean()

66507.5

In [12]:
df.to_csv("./output.csv",index=False)

### Calculate weekly costs

In [31]:
weekly=0
weekly_record=[]
for x in range(0,len(df['Total Cost'])):
    if x%7!=0:
        weekly+=df.loc[x,'Total Cost']
    else:
        if x==0:
            weekly+=df.loc[x,'Total Cost']
        else:
            weekly_record.append(weekly)
            weekly=0
            weekly+=df.loc[x,'Total Cost']

In [32]:
weekly=pd.DataFrame(data=weekly_record, columns=['Total Cost by Week'])

In [33]:
weekly.head()

,Total Cost by Week
0,2415000
1,1722000
2,2125000
3,2222000
4,2128000


In [37]:
annual=0
annual_record=[]

for x in range(0,len(weekly)):
    if x%52!=0:
        annual+=weekly.loc[x,'Total Cost by Week']
    else:
        if x==0:
            annual+=weekly.loc[x,'Total Cost by Week']
        else:
            annual_record.append(annual)
            annual=0
            annual+=weekly.loc[x,'Total Cost by Week']

In [39]:
annual=pd.DataFrame(data=annual_record, columns=['Total Cost by Year'])

In [40]:
annual.head()

,Total Cost by Year
0,111172000
1,111146000
2,110121000
3,110139000
4,109801000


In [43]:
annual.mean()

Total Cost by Year    1.107233e+08
dtype: float64

In [44]:
annual.to_csv("./annual_costs.csv",index=False)

### Calculate monthly demurrage

In [34]:
monthly_demurrage=0
months=[31,28,31,30,31,30,31,31,30,31,30,31]
month=0
count=0
demurrage_tracker=[]

for x in range(0,len(df['Total Cost'])):
    if count==months[month]:
        demurrage_tracker.append(monthly_demurrage)
        count=0
        monthly_demurrage=0
        if month==11:
            month=0
        else:
            month+=1    
    else:
        monthly_demurrage+=df.loc[x,'Demurrage Fees']
        count+=1

### Save monthly costs to dataframe

In [35]:
monthly=pd.DataFrame(data=demurrage_tracker, columns=['Monthly Demurrage'])

In [36]:
monthly.head()

,Monthly Demurrage
0,2070000
1,1750000
2,1670000
3,2080000
4,2250000


In [19]:
monthly.describe()

,Monthly Demurrage
count,3.180000e+02
mean,2.021855e+06
std,2.420920e+05
min,1.410000e+06
25%,1.845000e+06
50%,2.027500e+06
75%,2.190000e+06
max,2.805000e+06


In [20]:
monthly.mean()

Monthly Demurrage    2.021855e+06
dtype: float64

In [21]:
monthly.to_csv("./monthly_demurrage.csv",index=False)